In [1]:
# Generate video for a program. Make sure you have the executable open
import sys
import os
import re
import json
import glob
import copy
import time
import random
import shutil
from pathlib import Path

sys.path.append('../simulation/')
sys.path.append('../dataset_utils/')

import add_preconds
import augmentation_utils
import evolving_graph.check_programs as check_programs
import evolving_graph.utils as utils

from utils_demo import *
from unity_simulator.comm_unity import UnityCommunication
from unity_simulator import utils_viz

equiv_class = utils.load_name_equivalence()
prefab_class = utils.load_name_equivalence(file_name='../../resources/PrefabClass.json')

rooms = ["bedroom", "livingroom", "kitchen", "bathroom"]

# class_name_equivalenceにあるかもしれませんが
# graphに実際存在している名前
alternative_temp_obj = {
    "remote_control" : "remotecontrol",
    "washing_machine" : "washingmachine",
    "clothes_pile" : "clothespile",# basket_for_clothes
    "pants" : "clothespants",# clothes_pants
    "shirt" : "clothesshirt",# clothes_shirt
    "electrical_outlet" : "powersocket",
    "kitchen_cabinets" : "kitchencabinet",# cupboard
    "cutlery_fork" : "cutleryfork",# fork
    "cutlery_knife" : "cutleryknife",# knife
    "kitchen_table" : "kitchentable",
    "coffee_table" : "coffeetable",
    "diningtable" : "kitchentable",# table
    "wall_phone" : "wallphone",
    "lamp" : "tablelamp",# floor_lamp
    "dish_bowl" : "dishbowl",# bowl
    "bottle_water" : "bottlewater",# water
    "kitchen_counter" : "kitchencounter",
    "lotionbottle" : "hairproduct",# shampoo
    "washing_sponge" : "washingsponge",# sponge
    "picture" : "wallpictureframe",# picture
    "rag" : "towel",
    "bread" : "breadslice",# food_bread
    "garbage_can" : "garbagecan",
    "papertowel" : "napkin",# paper_towel
}

# pythonのcheck_scriptがid_mappingを探す時、変わったclass_name
change_execute_map = {'lightswitch':'light', 'tv':'television', 'remotecontrol':'controller', 'sofa':'couch'}
id_map_used_obj = ['lightswitch'] # id_mappingを使う必要があるオブジェクト　＊＊＊　将来にもずっと更新する必要　＊＊＊
room_map = {'home_office':'livingroom', 'living_room':'livingroom', 'entrance_hall':'livingroom',
            'kitchen':'kitchen', 'dining_room':'kitchen',
            'bathroom':'bathroom', 
            'bedroom':'bedroom', 'kids_bedroom':'bedroom'}

fixed_cameras = {
    'kitchen':{1:[63,64,65,66], 2:[39,40,41,42], 3:[28,29,30,31], 4:[30,31,32,33], 5:[49,50,51,52], 6:[61,62,63,64], 7:[41,42,43,44]},
    'bathroom':{1:[13,14,15,16], 2:[9,10,11,12], 3:[79,80,81,82], 4:[45,46,47,48], 5:[80,81,82,83], 6:[12,13,14,15], 7:[11,12,13,14]},
    'bedroom':{1:[38,39,40,41], 2:[60,61,62,63], 3:[97,98,99,100], 4:[65,66,67,68], 5:[64,65,66,67], 6:[34,35,36,37], 7:[89,90,91,92]},
    'livingroom':{1:[86,87,88,89], 2:[82,83,84,85], 3:[47,48,49,50], 4:[91,92,93,94], 5:[16,17,18,19], 6:[84,85,86,87], 7:[67,68,69,70]}
}

In [2]:
def get_program_file(file_path):
    program_name = ''
    description = ''
    list_of_steps = []
    with open(file_path, "r") as f:
        lines = f.readlines()
        program_name = lines[0].replace("\n", "")
        description = lines[1].replace("\n", "")
        for line in lines[2:]:
            line = line.replace("\n", "")
            if line.startswith("["):
                list_of_steps.append(line)
            else:
                pass

    return program_name, description, list_of_steps

def switchOff(g, script_lines):
    off_list = ['tv', 'television', 'computer', 'faucet']
    for class_name in off_list:
        nodes = find_nodes(g, class_name=class_name)
        for node in nodes:
            states = [x for x in node['states'] if x != "ON"]
            if "OFF" not in states:
                states.append("OFF")
            node['states'] = states
    
    is_first_switchon = True
    is_first_switchoff = True
    for line in script_lines:
        action, objs, indexs = augmentation_utils.parseStrBlock(line)
        if action == 'SWITCHOFF' and objs[0] in ['light', 'lightswitch']:
            if is_first_switchoff:
                node = find_nodes(g, id=int(indexs[0]))[0]
                states = node['states']
                if 'OFF' in states:
                    states.remove('OFF')
                    states.append('ON')
                elif 'ON' not in states:
                    states.append('ON')
                node['states'] = states
            is_first_switchon = False
        elif action == 'SWITCHON' and objs[0] in ['light', 'lightswitch']:
            if is_first_switchon:
                node = find_nodes(g, id=int(indexs[0]))[0]
                states = node['states']
                if 'ON' in states:
                    states.remove('ON')
                    states.append('OFF')
                elif 'OFF' not in states:
                    states.append('OFF')
                node['states'] = states
            is_first_switchoff = False
            comm.expand_scene(graph)
    return g

#ランダムでid2000以降のobjectを追加してしまうので、削除する
def delete2000(graph_state_list):
    new_graph_state_list = []
    for graph_state in graph_state_list:
        new_nodes = [x for x in graph_state["nodes"] if x["id"] < 2000]
        new_edges = [x for x in graph_state["edges"] if x["from_id"] < 2000 and x["to_id"] < 2000]
        new_graph_state_list.append({"nodes": new_nodes, "edges": new_edges})
    return new_graph_state_list

def final_json_update(unity_json_dict, python_json_dict):
    try:
        new_graph = copy.deepcopy(unity_json_dict)
        for new_node in new_graph["nodes"]:
            new_node_id = new_node["id"]
            for graph_state_node in python_json_dict["nodes"]:
                if new_node_id == graph_state_node["id"]:
                    new_node["states"] = []
                    new_node["states"] = graph_state_node["states"]
                    new_node["properties"] = graph_state_node["properties"]
           
    except Exception as e:
        print(e)
    
    return new_graph

def export(script_lines, file_name, activity_name, description, graph_state_list, obj_id_map):
    file_path = "../simulation/Output/scene"+str(scene_id)+"/graph states/"
    if os.path.exists(file_path) == False:
        os.mkdir(file_path)
    file_path = file_path + activity_name + "/"
    if os.path.exists(file_path) == False:
        os.mkdir(file_path)

    script_path = file_path + "activityList-program.txt"
    with open(script_path, 'w') as f:
        for s in script_lines:
            s = "<char0> " + s
            f.write("%s\n" % s)

    description_path = file_path + "program-description.txt"
    with open(description_path, 'w') as f:
        f.write("%s\n" % activity_name)
        f.write("%s\n" % description)

    program_path = file_path + "executed-program.txt"
    with open(program_path, 'w') as f:
        f.write("%s\n" % activity_name)
        f.write("%s\n\n\n" % description)
        for s in script_lines:
            s = "<char0> " + s
            f.write("%s\n" % s)
    
    action_list = []
    duration_list = []
    output_graph_map = {}
    new_graph_state_list = []
    new_graph_state_list.append(copy.deepcopy(graph_state_list[0]))
    for script_line in script_lines:
        action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
        action_list.append(action)

    output_graph_path = "../simulation/Output/scene"+str(scene_id)+"/" + file_name + "/0/Action_????_0_graph.json"
    for graph_path in sorted(glob.glob(output_graph_path)):
        temp_path = graph_path.replace("../simulation/Output/scene"+str(scene_id)+"/"+file_name+"/0\\", "")
        frame_times = int(temp_path.split("_")[1])
        output_graph_map[frame_times] = graph_path

    ftaa_path = "../simulation/Output/scene"+str(scene_id)+"/" + file_name + "/0/ftaa_" + file_name + ".txt"
    with open(ftaa_path, "r") as f:
        ftaa_lines = f.readlines()
        # sometimes FIND don't print out in ftaa file if the FIND object is near avatar
        if 'FIND' in action_list and len(ftaa_lines) < len(action_list):
            action_list.remove('FIND')
        i = 0
        graph_state_index = 1
        for action_index, action in zip(range(len(action_list)), action_list):
            ftaa_index = int(ftaa_lines[i].split(" ")[0])
            ftaa_action = ftaa_lines[i].split(" ")[1]
            if action_index == ftaa_index and action == ftaa_action:
                start_frame = ftaa_lines[i].split(" ")[2]
                end_frame = ftaa_lines[i].split(" ")[3]
                i += 1
            elif i+1 < len(ftaa_lines): # look for the action in the next one line of ftaa file
                start_frame = ftaa_lines[i].split(" ")[2]
                ftaa_index = int(ftaa_lines[i+1].split(" ")[0])
                if action_index == ftaa_index: # ftaa has two lines for an action
                    if i+2 < len(ftaa_lines): # look for the action in the next two lines of ftaa file
                        ftaa_index = int(ftaa_lines[i+2].split(" ")[0])
                        if action_index == ftaa_index: # ftaa has three lines for an action
                            end_frame = ftaa_lines[i+2].split(" ")[3]
                            i += 3
                        else:
                            end_frame = ftaa_lines[i+1].split(" ")[3]
                            i += 2
                    else:
                        end_frame = ftaa_lines[i+1].split(" ")[3]
                        i += 2
                else:
                    end_frame = ftaa_lines[i].split(" ")[3]
                    i += 1
            else:
                start_frame = ftaa_lines[i].split(" ")[2]
                end_frame = ftaa_lines[i].split(" ")[3]
                i += 1
            duration_frame = int(end_frame) - int(start_frame)
            duration = duration_frame * 0.033
            duration_list.append(str(duration))
            
            global graph_output_interval
            frame_counter = 0
            while (True):
                check_value = int(end_frame) - frame_counter
                if check_value < graph_output_interval:
                    if action_index == (len(action_list)-1) and int(end_frame) == frame_counter:
                        frame_counter -= graph_output_interval
                    with open(output_graph_map[frame_counter], 'r', encoding='utf-8-sig', errors='ignore') as json_file:
                        g = final_json_update(json.load(json_file), graph_state_list[graph_state_index])
                        new_graph_state_list.append(copy.deepcopy(g))
                    graph_state_index += 1
                    json_file.close()
                    break
                else:
                    frame_counter += graph_output_interval

    new_ftaa_path = file_path + "/ftaa_" + file_name + ".txt"
    shutil.copyfile(ftaa_path, new_ftaa_path)

    duration_path = file_path + "duration.txt"
    with open(duration_path, 'w') as f:
        for d in duration_list:
            f.write("%s\n" % d)
    
    file_index = 0
    for graph in new_graph_state_list:
        file_index += 1
        json_path = file_path + "activityList-graph-state-" + '{0:03d}'.format(file_index) + ".json"
        with open(json_path, 'w') as outfile:
            json.dump(graph, outfile, indent=2)
        json_path = ''

In [3]:
def replace_index(script_line, replace_value1, replace_value2=None):
    start_indexes = [i.start() for i in re.finditer('\(', script_line)]
    end_indexes = [i.start() for i in re.finditer('\)', script_line)]
    if replace_value2 is not None and len(start_indexes) > 1:
        script_line = script_line[:start_indexes[1]+1] + replace_value2 + script_line[end_indexes[1]:]
    script_line = script_line[:start_indexes[0]+1] + replace_value1 + script_line[end_indexes[0]:]
    return script_line

def get_obj_name(current_obj):
    graph_class_names = [node['class_name'] for node in graph['nodes']]
    # find in graph for equaivalence objs
    for equ_new_obj in equiv_class[current_obj]:
        if alternative_temp_obj.get(equ_new_obj) is not None:
            equ_new_obj = alternative_temp_obj[equ_new_obj]
        if equ_new_obj in graph_class_names:
            new_obj = equ_new_obj
            break
        else:
            new_obj = equiv_class[current_obj][0]
    return new_obj

# graphにない場合
def replace_obj_and_get_index(script_line, current_obj):
    # class_name_equivalenceにない場合
    if (equiv_class.get(current_obj) is None):
        script_line, obj_index = get_obj_index_from_alternative(script_line, current_obj)
    else:
        new_obj = get_obj_name(current_obj)

        if (alternative_temp_obj.get(new_obj) is None):
            script_line = script_line.replace(current_obj, new_obj)
            action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
            obj_index = get_obj_index(new_obj, action)
            if (obj_index == '1'):
                add_node_and_relation(new_obj)
                obj_index = get_obj_index(new_obj, action)
        else:
            # equivalenceの値がalternativeにある場合
            script_line, obj_index = get_obj_index_from_alternative(script_line, new_obj)
    return script_line, obj_index

def get_obj_index_from_alternative(script_line, current_obj):
    action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
    if (alternative_temp_obj.get(current_obj) is not None):
        new_obj = alternative_temp_obj[current_obj]
        script_line = script_line.replace(current_obj, new_obj)
        obj_index = get_obj_index(new_obj, action)
        if (obj_index == '1'):
            # Add unfound object in the entire scene
            add_node_and_relation(new_obj)
            obj_index = get_obj_index(new_obj, action)
    else:
        if (current_obj in (name['className'].lower() for name in prefab_class['prefabClasses'])):
            add_node_and_relation(current_obj)
            obj_index = get_obj_index(current_obj, action)
        else:
            script_line = None
            obj_index = '1'
    return script_line, obj_index

def get_obj_index(obj, action):
    id = [node['id'] for node in graph['nodes'] if node['class_name'] == obj]
    edges = graph['edges']
    if len(id) > 0:
        global current_room
        if obj in rooms:
            current_room = obj
            return str(id[0])
        else:
            current_room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == current_room][0]
            ids_in_room = [edge['from_id'] for edge in edges if edge['relation_type'] == 'INSIDE' and edge['to_id'] == current_room_id]
            obj_ids_in_room = [x for x in id if x in ids_in_room]
            if len(obj_ids_in_room) > 0: #現在の部屋にオブジェクトがある
                return str(obj_ids_in_room[0])
            else: #別の部屋に移動する・オブジェクトを探す
                return str(id[0])
    else:
        return '1'


def add_object_out_of_script(obj, room, place, relation_type, place_obj_index):
    global scene_id
    max_id = max(x['id'] for x in graph['nodes']) + 1
    room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == room][1 if (scene_id == 3 and room == 'bedroom') else 0] # change index 0 to 1 when scene 3's bedroom
    ids_in_room = [edge['from_id'] for edge in graph['edges'] if edge['relation_type'] == 'INSIDE' and edge['to_id'] == room_id]
    possible_place_id = [node['id'] for node in graph['nodes'] if node['class_name'] == place]
    place_id = [x for x in possible_place_id if x in ids_in_room][place_obj_index]
    add_node(graph, {'class_name': obj, 
                        'category': '', 
                        'id': max_id, 
                        'properties': [], 
                        'states': []})
    add_edge(graph, max_id, 'INSIDE', room_id)
    add_edge(graph, max_id, relation_type, place_id)
    print(obj, max_id, 'is added to the scene on the ', place, 'inside ', room)
    comm.expand_scene(graph)

# Need to change room name, place name, relation type, place index according to the activity requirements.
# This function is called when there is no activity's object in the all room.
def add_node_and_relation(obj):
    if (obj not in (x['class_name'] for x in graph['nodes'])):
        add_object_out_of_script(obj, 'kitchen', 'kitchentable', 'ON', 0)

In [4]:
def correct_script(script_list):
    result_list = []
    for script_line in script_list:
        action, objs, indexs = augmentation_utils.parseStrBlock(script_line)

        # アクションのみ場合
        if len(objs) == 0:
            result_list.append(script_line)
            continue
        elif indexs[0].isdigit() and indexs[0] != '1': ## Only for accurate ID indication
            if (objs[0] in change_execute_map):
                change_obj_name = change_execute_map[objs[0]]
                keyword = (change_obj_name, int(indexs[0]))
            else:
                keyword = (objs[0], int(indexs[0]))
            if obj_id_map.get(keyword) is None:
                obj_id_map[keyword] = int(indexs[0])
            
            graph_class_names = [node['class_name'] for node in graph['nodes']]
            if objs[0] not in graph_class_names:
                new_obj = get_obj_name(objs[0])
                script_line = script_line.replace(objs[0], new_obj)
            if len(objs) > 1 and objs[1] not in graph_class_names:
                new_obj = get_obj_name(objs[1])
                script_line = script_line.replace(objs[1], new_obj)
                
            result_list.append(script_line)
        else:
            first_obj = objs[0]
            if len(objs) > 1:
                second_obj = objs[1]
            else:
                second_obj = None
            # ①graphに探す→②class_name_equivalenceに探す→③alternative_temp_objに探す
            first_obj_index = get_obj_index(first_obj, action)
            if first_obj_index == '1':
                script_line, first_obj_index = replace_obj_and_get_index(script_line, first_obj)
            if second_obj is not None and script_line is not None:
                second_obj_index = get_obj_index(second_obj, action)
                if second_obj_index == '1':
                    script_line, second_obj_index = replace_obj_and_get_index(script_line, second_obj)
            else:
                second_obj_index = None

            if script_line is not None:
                script_line = replace_index(script_line, first_obj_index, second_obj_index)
                action, objs, indexs = augmentation_utils.parseStrBlock(script_line)
                if (objs[0] in change_execute_map):
                    change_obj_name = change_execute_map[objs[0]]
                    keyword = (change_obj_name, int(indexs[0]))
                else:
                    keyword = (objs[0], int(indexs[0]))
                if obj_id_map.get(keyword) is None:
                    obj_id_map[keyword] = int(indexs[0])
                result_list.append(script_line)
    return result_list

def available_fixed_camera(room_name):
    mode_list = ["PERSON_FROM_BACK", "AUTO"]
    mode_map = fixed_cameras[room_name]
    return mode_list + mode_map[scene_id]

In [11]:
scene_id = 3
program_files = []
data_path = "../dataset/Test/scene"+str(scene_id)+"/*.txt"
for file_path in sorted(glob.glob(data_path)):
    file_name = file_path.replace(file_path[0:file_path.rfind("\\")+1], "")
    
    program_name, description, list_of_steps = get_program_file(file_path)
    program_files.append({
        "file_name":file_name,
        "name": program_name,
        "description": description,
        "list_of_steps": list_of_steps
    })

In [ ]:
comm = UnityCommunication(timeout_wait=10800)

for program in program_files:
    print('\nStarting',program["file_name"])
    message = comm.reset(scene_id - 1)
    su, graph = comm.environment_graph()
    program_name = program["name"]
    script_lines = program["list_of_steps"]

    # the following is the object adding code to environment and it has to be used here not other places.
    # if you need more than one object, you can use several times
    # the purpose of this line is to add new object even the same object is existed in same room or any other different room
#     add_object_out_of_script('wine', 'kitchen', 'kitchentable', 'ON', 0)
    add_object_out_of_script('paper', 'bedroom', 'desk', 'ON', 0)

    initial_room = 'bedroom' # Initial Room : Need to change according to activity　＊＊＊＊＊＊＊
    current_room = initial_room
    obj_id_map = {}
    graph_output_interval = 5 # Control your output graph state json frequency
    script_lines = correct_script(script_lines)
    graph = switchOff(graph, script_lines)
    comm.add_character(initial_room=initial_room)
    su, graph = comm.environment_graph()

    try:
        preconds = add_preconds.get_preconds_script(script_lines).printCondsJSON()
    except Exception as ex:
        print(ex)
        continue

    print('Preconds\n',preconds,'\n')

    modified_graph = check_programs.translate_graph_dict_nofile(graph)
    is_id_map_used = False
    if len([k[0] for k,v in obj_id_map.items() if k[0] in id_map_used_obj]) > 0:
        is_id_map_used = True
    info = check_programs.check_script(script_lines, preconds, graph_path=None, inp_graph_dict=modified_graph, id_mapping=obj_id_map if is_id_map_used else {})
    message, final_state, graph_state_list, graph_dict, id_mapping, info, helper, executed_script = info
    graph_state_list = delete2000(graph_state_list)
    print(message)

    if message != 'Script is executable':
        print(message)
        continue

    executed_lines =[]
    for line in script_lines:
        line_to_execute = '<char0> '+line
        executed_lines.append(line_to_execute)
        
    sus_msg, original_graph = comm.environment_graph()
    graph_state_list[0] = final_json_update(original_graph, graph_state_list[0])

    s, message = comm.render_script(
        executed_lines,
        recording=True, 
        find_solution=False, 
        frame_rate=30,
        save_pose_data=True,
        vis_check_object=True, 
        vis_check_character=True,
        vis_check_object_all=True,
        out_graph=True,
        per_frame=graph_output_interval,
        camera_mode=available_fixed_camera(current_room),
        file_name_prefix=program["file_name"][:-4],
        output_folder="Output/scene" + str(scene_id))
    print(str(s) + (str(": " + str(message)) if s==False else ''))

    export(script_lines, program["file_name"][:-4], program['name'], program['description'], graph_state_list, obj_id_map)


Starting Write_notes.txt
paper 424 is added to the scene on the  desk inside  bedroom
Preconds
 [{'grabbed': ['paper', '424']}, {'free': ['chair', '405']}] 

Script is executable


In [ ]:
#### complementing missing images
try:
    for x in range(6):
        for program in program_files:
            program_file_name = program["file_name"][:-4]
            img_path = "../simulation/Output/scene"+str(scene_id) + "/" + program_file_name + "/0/??????_????_" + str(x) + "_normal.png"
            i = 0
            for file_path in sorted(glob.glob(img_path)):
                file_name = file_path.replace("../simulation/Output/scene"+str(scene_id) + "/" + program_file_name + "/0\\", "")
                file_num = file_name.split("_")[1]
                if i != int(file_num):
                    print(str(i) + " " + str(int(file_num)))
                    img = PIL.Image.open(file_path)
                    new_file_name = "../simulation/Output/scene"+str(scene_id) + "/" + program_file_name + "/0/" + "Action_%04d_" + str(x) + "_normal.png"
                    img.save(new_file_name % i)
                    print(file_name)
                    print(file_name + " - 1")
                    print("save")
                    i += 1
                i += 1
except Exception as e:
    print(e)

In [ ]:
import subprocess

dir_path = "../simulation/Output/scene"+str(scene_id)+"/"
output_path = dir_path + "movies/"
if os.path.exists(output_path) == False:
    os.mkdir(output_path)
try:
    for x in range(6):
        for activity in program_files:
            file_name = activity["file_name"]
            activity_name = activity["name"]
            file_name = file_name[:-4]
            if os.path.exists(dir_path + file_name) == True:
                sh = subprocess.check_output(["ffmpeg", "-framerate", "14.5", "-i", dir_path + file_name + "/0/Action_%04d_" + str(x) + "_normal.png", "-vcodec", "libx264", "-pix_fmt", "yuv420p", "-r", "30", output_path + activity_name + "_" + str(x) + ".mp4"])
                print(sh)

            img_json_path = dir_path + file_name + "/0/"
            seperate_file_path = img_json_path + activity_name + "_" + str(x) + "/"
            if os.path.exists(seperate_file_path) == False:
                os.mkdir(seperate_file_path)
            desired_files = [fn for fn in os.listdir(img_json_path) if "_"+str(x)+"_" in fn and "graph.json" not in fn]
            for each_file in desired_files:
                shutil.move(img_json_path+each_file, seperate_file_path)
            if x != 0:
                removing_files = [fn for fn in os.listdir(img_json_path) if "_"+str(x)+"_graph" in fn]
                for each_file in removing_files:
                    os.remove(img_json_path+each_file)
except Exception as e:
    print(e)

In [6]:
# Ignore this block, this is only for reason of searching graph
sceneNo = 3
room_name = 'bedroom'
def searchInGraph (sceneNo, test_room_name):
    comm = UnityCommunication()
    message = comm.reset(sceneNo-1)
    su, graph = comm.environment_graph()
    if test_room_name == '':
        return graph
    else:
        test_room_id = [node['id'] for node in graph['nodes'] if node['class_name'] == test_room_name][1 if (sceneNo == 3 and test_room_name == 'bedroom') else 0]
        test_ids_in_room = [edge['from_id'] for edge in graph['edges'] if edge['relation_type'] == 'INSIDE' and edge['to_id'] == test_room_id]
        filtered_graph = [node for node in graph['nodes'] if node['id'] in test_ids_in_room]
        print(max(x['id'] for x in graph['nodes']) + 1)
        return filtered_graph
# out_file = open("../simulation/Output/scene"+str(sceneNo)+".json", "w")  
# json.dump(graph, out_file, indent = 6)   
# out_file.close()
print(searchInGraph(sceneNo, room_name))

424


[{'id': 359, 'category': 'Floor', 'class_name': 'floor', 'prefab_name': 'PRE_FLO_Wood_01_04', 'obj_transform': {'position': [6.25, 0.0, 3.74], 'rotation': [0.0, 0.7071068, 0.0, 0.7071067], 'scale': [1.0, 1.0, 1.0]}, 'bounding_box': {'center': [6.25, 0.0, 4.99], 'size': [2.5, 9.049975e-06, 2.5]}, 'properties': ['SURFACES'], 'states': []}, {'id': 360, 'category': 'Floor', 'class_name': 'floor', 'prefab_name': 'PRE_FLO_Wood_01_04', 'obj_transform': {'position': [6.25, 0.0, 1.24000025], 'rotation': [0.0, 0.7071068, 0.0, 0.7071067], 'scale': [1.0, 1.0, 1.0]}, 'bounding_box': {'center': [6.24999952, 0.0, 2.49000025], 'size': [2.5, 9.049975e-06, 2.5]}, 'properties': ['SURFACES'], 'states': []}, {'id': 361, 'category': 'Floor', 'class_name': 'floor', 'prefab_name': 'PRE_FLO_Wood_01_04', 'obj_transform': {'position': [6.24999952, 0.0, -1.25999951], 'rotation': [0.0, 0.7071068, 0.0, 0.7071067], 'scale': [1.0, 1.0, 1.0]}, 'bounding_box': {'center': [6.24999952, 0.0, -0.009999633], 'size': [2.5, 9

In [ ]:
import csv

for x in range(1, 8):
    scene_index = x
    room = ''
    graph_data = searchInGraph(scene_index, room)
    with open('../simulation/Output/scene'+str(scene_index)+'.csv', 'wt', newline='') as out_file:
        csv_writer = csv.writer(out_file, delimiter=',', dialect="excel")
        csv_writer.writerow(['ID', 'Class Name'])
        for node in graph_data['nodes']:
            csv_writer.writerow([node['id'], node['class_name']])
        